<a href="https://colab.research.google.com/github/twalker47/helloAI/blob/main/Assignment8_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# mount google drive here
#!pip uninstall torch_xla

!pip install --upgrade cloud-tpu-client

!pip install -i "https://pip.repos.neuron.amazonaws.com" torch_xla

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.0
    Uninstalling google-api-core-2.11.0:
      Successfully uninstalled google-api-core-2.11.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

In [6]:
# create and train a pytorch digit classifier
# source: chatgpt with the prompt "create a pytorch mnist digit classifier"
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

import os
os.makedirs(os.path.dirname(PATH), exist_ok=True)

# Set a flag to indicate whether we are (re)-training the model
# or loading an already trained model
PATH = "/content/drive/MyDrive/models/classifier-digits.pt"
#loadmodel = False

# Set device to either CUDA or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.dropout(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

if loadmodel:
  model = Net()
  model.load_state_dict(torch.load(PATH))
  model = model.to(device)
  model.eval()

else:
  # Create an instance of the model and send it to the device
  model = Net().to(device)

  # Load the MNIST dataset and apply transformations
  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.1307,), (0.3081,))
  ])

  train_set = datasets.MNIST('../data', train=True, download=True, transform=transform)
  test_set = datasets.MNIST('../data', train=False, download=True, transform=transform)

  # Set batch size and create data loaders
  batch_size = 64

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)


  # Define the optimizer and loss function
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
  criterion = nn.CrossEntropyLoss()

  # Train the model
  num_epochs = 2

  for epoch in range(num_epochs):
      model.train()
      for batch_idx, (data, target) in enumerate(train_loader):
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))

      model.eval()
      test_loss = 0
      correct = 0
      with torch.no_grad():
          for data, target in test_loader:
              data, target = data.to(device), target.to(device)
              output = model(data)
              test_loss += criterion(output, target).item()
              pred = output.argmax(dim=1, keepdim=True)
              correct += pred.eq(target.view_as(pred)).sum().item()

      test_loss /= len(test_loader.dataset)
      print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))  

  torch.save(model.state_dict(), PATH)


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.347908
Train Epoch: 0 [6400/60000 (11%)]	Loss: 1.472815
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.358262
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.300413
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.248143
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.213565
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.401892
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.241117
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.297031
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.193586
Test set: Average loss: 0.0024, Accuracy: 9524/10000 (95%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.295638
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.348853
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.055743
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.296754
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.213590
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.109081
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.096402
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.194746
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.158891
Tr

In [2]:
#OK now we have a good model for analyzing the output from our GAN
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html

# flag for saving/loading
PATH_G = "/content/drive/MyDrive/models/generator-digits.pt"
PATH_D = "/content/drive/MyDrive/models/discriminator-digits.pt"
loadgan = False

# Set random seed for reproducibility
torch.manual_seed(42)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.fc = nn.Linear(100, 7 * 7 * 128)
        self.conv1 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1)

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 128, 7, 7)
        x = self.relu(self.conv1(x))
        x = self.tanh(self.conv2(x))
        return x

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.fc = nn.Linear(7 * 7 * 128, 1)

        self.relu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, 7 * 7 * 128)
        x = self.sigmoid(self.fc(x))
        return x

# Instantiate the generator and discriminator networks
generator = Generator()
discriminator = Discriminator()

if loadgan:
  generator.load_state_dict(torch.load(PATH_G))
  generator = generator.to(device)
  generator.eval()
  discriminator.load_state_dict(torch.load(PATH_D))
  discriminator = discriminator.to(device)
  discriminator.eval()

else:
  # Define the loss function and optimizer
  criterion = nn.BCELoss()
  generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
  discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

  # Load the dataset
  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.5,), (0.5,))
  ])
  dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
  dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

  # Define the number of epochs and the device to use
  num_epochs = 10
  device = torch.device('xla')

  # Move the generator and discriminator networks to the device
  generator.to(device)
  discriminator.to(device)

  # Train the generator and discriminator networks
  time_total = 0
  d_losses = []
  g_losses = []
  counts = [0]*10
  confidences = []
  for epoch in range(num_epochs):
      time_epoch = 0
      start = time.time()
      for i, (real_images, _) in enumerate(dataloader):
          batch_size = real_images.shape[0]
          real_labels = torch.ones(batch_size, 1, device=device)
          fake_labels = torch.zeros(batch_size, 1, device=device)

          # Train the discriminator network
          discriminator_optimizer.zero_grad()
          real_images = real_images.to(device)
          real_outputs = discriminator(real_images)
          real_loss = criterion(real_outputs, real_labels)

          noise = torch.randn(batch_size, 100, device=device)
          fake_images = generator(noise)
          fake_outputs = discriminator(fake_images.detach())
          fake_loss = criterion(fake_outputs, fake_labels)

          discriminator_loss = real_loss + fake_loss
          discriminator_loss.backward()
          discriminator_optimizer.step()

          # Train the generator network
          generator_optimizer.zero_grad()
          noise = torch.randn(batch_size, 100, device=device)
          fake_images = generator(noise)
          fake_outputs = discriminator(fake_images)
          generator_loss = criterion(fake_outputs, real_labels)
          generator_loss.backward()
          generator_optimizer.step()

          # Print the loss statistics
          if i % 100 == 0:
              print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{i+1}/{len(dataloader)}] '
                    f'Discriminator Loss: {discriminator_loss.item():.4f} '
                    f'Generator Loss: {generator_loss.item():.4f}')

      # Generate 100 samples and count how many of each digit 
      with torch.no_grad():
          # Define the fixed noise vector for generating samples
          sample_noise = torch.randn(100, 100, device=device)
          generated_images = generator(sample_noise)
          generated_images = 0.5 * (generated_images + 1)
          output = model(generated_images)
          pred = output.argmax(dim=1, keepdim=True)
          grid = make_grid(generated_images[:16].cpu(), nrow=8, pad_value=1)
          plt.imshow(grid.permute(1, 2, 0))
          plt.axis('off')
          plt.title([int(pred[x]) for x in range(16)])
          plt.show()
          #print(torch.exp(output[0])) # torch.exp converts the output back into regular softmax probabilities (confidences)
          avg_conf = torch.mean(torch.exp(output).max(dim=1).values)
          confidences.append(avg_conf)
          for p in pred:
              counts[p] += 1

      g_losses.append(generator_loss.item())
      d_losses.append(discriminator_loss.item())
      time_epoch = time.time() - start
      time_total += time_epoch
      print(f"average prediction confidence over 100 predictions: {avg_conf}")
      print(f"prediction counts: {counts}")
      print(f"epoch time: {time_epoch}, total time: {time_total}")
        
      torch.save(generator.state_dict(), PATH_G)
      torch.save(discriminator.state_dict(), PATH_D)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html


RuntimeError: ignored

In [6]:
import matplotlib.pyplot as plt
import torch

with torch.no_grad():
  # Check if CUDA is available, otherwise use CPU
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # Define the fixed noise vector for generating samples
  sample_noise = torch.randn(100, 100, device=device)
  generated_images = generator(sample_noise)
  generated_images = 0.5 * (generated_images + 1)
  output = model(generated_images)
  pred = output.argmax(dim=1, keepdim=True)
  grid = make_grid(generated_images[:16].cpu(), nrow=8, pad_value=1)
  plt.imshow(grid.permute(1, 2, 0))
  plt.axis('off')
  plt.title([int(pred[x]) for x in range(16)])
  plt.show()
  #print(torch.exp(output[0])) # torch.exp converts th


NameError: ignored

In [4]:
import matplotlib.pyplot as plt

plt.plot(confidences)
plt.show()
plt.bar([f"x" for x in range(10)], counts)
plt.show()


NameError: ignored

In [7]:
# Import necessary libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(100, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 28*28)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.tanh(self.fc3(x))
        x = x.view(-1, 1, 28, 28)
        return x

# Define the classifier network
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# Initialize the generator and classifier networks
generator = Generator().to('cuda')
classifier = Classifier().to('cuda')

# Load the MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

# Define the optimizer and loss function for training the classifier
optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
criterion = nn.NLLLoss()

# Train the classifier for 5 epochs
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to('cuda'), data[1].to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Generate 100 images using the generator and classify them using the classifier
    with torch.no_grad():
        sample_noise = torch.randn(100, 100, device='cuda')
        generated_images = generator(sample_noise)
        generated_images = 0.5 * (generated_images + 1)
        output = classifier(generated_images)
        pred = output.argmax(dim=1, keepdim=True)

        # Count how many of each digit are generated
        counts = torch.zeros(10)
        for p in pred:
            counts[p.item()] += 1

        # Calculate the average prediction confidence
        avg_confidence = torch.exp(output).mean()

        # Spot-check several generated samples to see if they were


RuntimeError: ignored